In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

NameError: name 'os' is not defined

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

In [7]:
"""The purpose of this project is to make a self-playing Tetris game that is managed by a model that attempts to survive as long as possibele. Using six different tetris pieces, each with four different possible rientations, my objective is to clear out horizontal lines before letting the stack of blocks hit a certain height."""

'The purpose of this project is to make a self-playing Tetris game that is managed by a model that attempts to survive as long as possibele. Using six different tetris pieces, each with four different possible rientations, my objective is to clear out horizontal lines before letting the stack of blocks hit a certain height.'

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [8]:
"""Ultimately, this project differs greatly from others in many senses of the typicaly ML workflow. In thsi case, I am using reinforcement learning, so the data that I have is necessarily the times that I end up running a certain type of model. For now, the only data that I have access to are a few models that have been provided to me in the form of the already created Tetris game, making it so that I didn't have to code the whole framework myself for the GUI."""

"Ultimately, this project differs greatly from others in many senses of the typicaly ML workflow. In thsi case, I am using reinforcement learning, so the data that I have is necessarily the times that I end up running a certain type of model. For now, the only data that I have access to are a few models that have been provided to me in the form of the already created Tetris game, making it so that I didn't have to code the whole framework myself for the GUI."

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [9]:
"""I can't necessarily explore the given "dataset" per se, but I can still try to play the game to see what works and doesn't work best. By understanding this, I can see what types of features make a good model in comparison to a poorly performing one. So far, the repository we were given has 4 different models to test. A genetic algorithm which performs poorly, a random algorithm which also performs badly, a monte carlo tree search algorithm which doesn't perform that great, but finally, a greedy algorithm which performs very well. After trying these different models on the game, and visualizing them in Pycharm, I came to the conclusion that the files of board.py managed the board, piece.py managed the falling piece, and game.py tied everything together."""

'I can\'t necessarily explore the given "dataset" per se, but I can still try to play the game to see what works and doesn\'t work best. By understanding this, I can see what types of features make a good model in comparison to a poorly performing one. So far, the repository we were given has 4 different models to test. A genetic algorithm which performs poorly, a random algorithm which also performs badly, a monte carlo tree search algorithm which doesn\'t perform that great, but finally, a greedy algorithm which performs very well. After trying these different models on the game, and visualizing them in Pycharm, I came to the conclusion that the files of board.py managed the board, piece.py managed the falling piece, and game.py tied everything together.'

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [10]:
"""There isn't necessarily any data to be preprocessed, because the outcomes of the games or the gameplay itself isn't stored in a database. Once I actually begin modeling, though, using RL, I will have some data that can be managed."""

"There isn't necessarily any data to be preprocessed, because the outcomes of the games or the gameplay itself isn't stored in a database. Once I actually begin modeling, though, using RL, I will have some data that can be managed."

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


In [ ]:
import numpy as np
import random
from copy import deepcopy

from genetic_helpers import (
    get_peaks, get_holes, get_wells,
    get_bumpiness, get_row_transition, get_col_transition, bool_to_np
)
from board import Board
from piece import Piece, BODIES


class HybridTetrisAgent:
    def __init__(
        self,
        num_features=9,
        alpha=0.01,
        gamma=0.95,
        epsilon=1.0,
        epsilon_min=0.05,
        epsilon_decay=0.9997
    ):
        self.weights = np.random.uniform(-0.01, 0.01, num_features)
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay

    def extract_features(self, board_np):
        peaks = get_peaks(board_np)
        highest_peak = np.max(peaks)
        holes = get_holes(peaks, board_np)
        wells = get_wells(peaks)

        return np.array([
            np.sum(peaks),
            np.sum(holes),
            get_bumpiness(peaks),
            np.count_nonzero(np.count_nonzero(board_np, axis=0) == 0),
            np.max(wells),
            np.count_nonzero(np.array(holes) > 0),
            get_row_transition(board_np, highest_peak),
            get_col_transition(board_np, peaks),
            np.sum(np.count_nonzero(board_np, axis=1))
        ], dtype=float)

    def q_value(self, features):
        return float(np.dot(self.weights, features))

    def choose_action(self, board, piece):

        if random.random() < self.epsilon:
            rot_count = random.randint(0, 3)
            temp_piece = piece
            for _ in range(rot_count):
                temp_piece = temp_piece.get_next_rotation()
            max_x = board.width - len(temp_piece.skirt)
            x = random.randint(0, max_x)
            return x, temp_piece

        best_value = -1e18
        best_x = None
        best_piece = None

        rot_piece = piece
        for _ in range(4):
            rot_piece = rot_piece.get_next_rotation()
            max_x = board.width - len(rot_piece.skirt)

            for x in range(max_x + 1):
                try:
                    y = board.drop_height(rot_piece, x)
                except:
                    continue

                # simulate placement
                grid = deepcopy(board.board)
                for pos in rot_piece.body:
                    grid[y + pos[1]][x + pos[0]] = True

                grid_np = bool_to_np(grid)
                f = self.extract_features(grid_np)
                v = self.q_value(f)

                if v > best_value:
                    best_value = v
                    best_x = x
                    best_piece = rot_piece

        return best_x, best_piece

    def update(self, prev_features, reward, next_features):
        reward = np.clip(reward, -1, 1)
        prev_q = self.q_value(prev_features)
        next_q = self.q_value(next_features)
        if np.isnan(prev_q): prev_q = 0
        if np.isnan(next_q): next_q = 0

        td_target = reward + self.gamma * next_q
        td_error = td_target - prev_q
        td_error = np.clip(td_error, -5, 5)

        self.weights += self.alpha * td_error * prev_features
        self.weights = np.nan_to_num(self.weights)

        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)


def train_hybrid_agent(episodes=5000):
    agent = HybridTetrisAgent()
    print("Starting Hybrid Tetris training...")

    best_moves = 0
    last50_moves = []

    for episode in range(1, episodes + 1):
        board = Board()
        total_reward = 0
        moves = 0
        done = False

        while not done:
            piece = Piece(body=random.choice(BODIES)[0])
            prev_board_np = bool_to_np(board.board)
            prev_features = agent.extract_features(prev_board_np)

            x, piece_used = agent.choose_action(board, piece)
            if x is None or piece_used is None:
                done = True
                break

            # Place piece
            try:
                y = board.drop_height(piece_used, x)
                board.place(x, y, piece_used)
            except:
                done = True
                break

            cleared = board.clear_rows()
            next_board_np = bool_to_np(board.board)
            next_features = agent.extract_features(next_board_np)

            holes_next = np.sum(get_holes(get_peaks(next_board_np), next_board_np))
            if cleared > 0:
                reward = 1
            elif holes_next > 0:
                reward = -0.05
            else:
                reward = -0.01

            total_reward += reward

            agent.update(prev_features, reward, next_features)
            moves += 1

            if np.any(board.heights >= board.height):
                done = True
                
        last50_moves.append(moves)
        if len(last50_moves) > 50:
            last50_moves.pop(0)

        avg_last50 = np.mean(last50_moves)
        best_moves = max(best_moves, moves)

        print(
            f"Gen {episode:4d}/{episodes} | "
            f"Moves: {moves:4d} | "
            f"Best: {best_moves:4d} | "
            f"Avg50: {avg_last50:6.2f} | "
            f"Reward: {total_reward:7.3f} | "
            f"Eps: {agent.epsilon:.3f}"
        )

    np.save("trained_weights.npy", agent.weights)
    print("Training complete! Saved weights to trained_weights.npy")
    return agent


if __name__ == "__main__":
    train_hybrid_agent(episodes=1000)


"""Here is just the custom_model.py that I ended up using"""


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 
